In [1]:
import os
import numpy as np
import pandas as pd
from pycaret.regression import *
from os.path import exists

In [2]:
root="/home/sadagopan/amex-default-prediction/"
headers=pd.read_csv(root+"headers.csv")
headers = headers.columns
labels =pd.read_csv(root+"train_labels.csv")

In [3]:
samples = pd.read_csv(root + "sample_submission.csv")
#samples[samples['prediction'] == 0].count()[0]

In [13]:
predsrange = []
#predictions=pd.DataFrame()
train=True
test=True
#model = load_model(root+"/models/" + "lightgbm-x12-0123reg")
for i in range(0):
    try:
        #model = load_model(root+"/models/" + "x" + str(i) + "stackedreg")
        model = load_model(root+"/models/" + "lightgbm-x12-0123reg")
        test=True
    except:
        try:
            model = load_model(root+"/models/" + "lightgbm-x12-" + str(i) + "reg")
        except:
            try:
                model = load_model(root+"/models/" + "catboost-x" + str(i) + "reg")
            except:
                try:
                    model = load_model(root+"/models/" + "dt-x" + str(i) + "reg")
                except:
                    model = load_model(root+"/models/" + "et-x" + str(i) + "reg")

    if train:
        print("x" + str(i) + "13dts.csv", end=" ")
        testdf = (pd.read_csv("x" + str(i) + "13dts.csv"))
        preds = predict_model(model, data=testdf, verbose=True)
        preds[['customer_ID', 'target', 'Label']].to_csv(root+"/test/x"+ str(i) + ".preds", index=False)
        P = preds[preds['target']==1]['Label'].count()
        N = preds[preds['target']==0]['Label'].count()
        TP = preds[preds['target']==1][round(preds['Label']) == 1]['Label'].count()
        FP = preds[preds['target']==0][round(preds['Label']) == 1]['Label'].count()
        FN = preds[preds['target']==1][round(preds['Label']) == 0]['Label'].count()
        TN = preds[preds['target']==0][round(preds['Label']) == 0]['Label'].count()
        print((preds['Label'].min(), preds['Label'].max()), "TP", TP, "FP", FP, "FN", FN, "TN", TN,
              "total", preds['Label'].count(),
              "Prec:", round((TP / (TP + FP) * 100), 2) ,
             "Acc:",  round(((TP + TN) / (P + N))*100, 2) ,
             "F1:", round((2*TP / (2*TP + FP + FN))*100, 2))
        if len(predsrange) <= i:
            predsrange.append((preds['Label'].min(), preds['Label'].max()))
        else:
            predsrange[i] = (preds['Label'].min(), preds['Label'].max())
    if test:
        print("t" + str(i) + "13dts.csv", end=" ")
        testdf = (pd.read_csv("t" + str(i) + "13dts.csv"))
        preds = predict_model(model, data=testdf, verbose=True)
        preds['Labelnew'] = round(preds['Label'])
        labels = preds.groupby(['Labelnew'], as_index=False)['Labelnew'].count()
        print((preds['Label'].min(), preds['Label'].max()), " ".join([str(l) + ":" + str(labels.loc[l][0]) for l in labels.index]))
#        predictions = pd.concat([predictions, preds], ignore_index=True, axis=0)
        preds[['customer_ID', 'Label']].to_csv(root+"/test/t"+ str(i) + ".preds", index=False)

#model = load_model(root+"/models/" + "x11stackedreg")

#print((predsrange[i-2].min(), predsrange[i-2].max()))
#if len(predsrange) <= i:
#    predsrange.append((predsrange[i-2][0], predsrange[i-2][1]))
#else:
#    predsrange[i] = (predsrange[i-1][0], predsrange[i-1][1])

nof=4
xproc=[False, False, False, False, False, False]
#xproc=[False, False, False, False, False]
yproc=[True, True, True, True, True]
#yproc=[False, False, False, False, False]

collate=False
i=0
if not collate:
    xfiles = ["x12-" + str((i*nof)+j) + "dtcnt.csv" for i in range(int(20/nof)) for j in range(nof)]
    tfiles = ["t12-" + str(i) + "dtcnt.csv" for i in range(41)]
    for i in range(2, 5): #int(20/nof)):
        tag = "".join([str((i*nof)+j) for j in range(nof)])
        print("Loading...", root+"models/" + "lightgbm-x12-" + tag + "reg", end=" ")
        model = load_model(root+"/models/" + "lightgbm-x12-" + tag + "reg")
        if xproc[i]:
            for j in range(nof):
#                if exists(xfiles[i]):
                print(xfiles[((i*nof)+j)], end=" ")
                testdf = pd.read_csv(xfiles[((i*nof)+j)])
        #        else:
        #            print("x12-" + str((i*nof)-1) + "dtcnt.csv", end=" ")
        #            testdf = pd.read_csv("x12-" + str((i*nof)-1) + "dtcnt.csv")

                preds = predict_model(model, data=testdf, verbose=True)
                preds[['customer_ID', 'target', 'Label']].to_csv(root+"/test/" + tag + "/x12-" + str((i*nof)+j) + ".preds", index=False)
                P = preds[preds['target']==1]['Label'].count()
                N = preds[preds['target']==0]['Label'].count()
                TP = preds[preds['target']==1][round(preds['Label']) == 1]['Label'].count()
                FP = preds[preds['target']==0][round(preds['Label']) == 1]['Label'].count()
                FN = preds[preds['target']==1][round(preds['Label']) == 0]['Label'].count()
                TN = preds[preds['target']==0][round(preds['Label']) == 0]['Label'].count()
                print((preds['Label'].min(), preds['Label'].max()), "TP", TP, "FP", FP, "FN", FN, "TN", TN,
                      "total", preds['Label'].count(),
                      "Prec:", round((TP / (TP + FP) * 100), 2) ,
                     "Acc:",  round(((TP + TN) / (P + N))*100, 2) ,
                     "F1:", round((2*TP / (2*TP + FP + FN))*100, 2))
        preds= pd.DataFrame()
        for j in range(nof):
            preds = pd.concat([preds, pd.read_csv(root+"test/" + tag + "/x12-" + str((i*nof)+j) + ".preds")], ignore_index=True)
        P = preds[preds['target']==1]['Label'].count()
        N = preds[preds['target']==0]['Label'].count()
        TP = preds[preds['target']==1][round(preds['Label']) == 1]['Label'].count()
        FP = preds[preds['target']==0][round(preds['Label']) == 1]['Label'].count()
        FN = preds[preds['target']==1][round(preds['Label']) == 0]['Label'].count()
        TN = preds[preds['target']==0][round(preds['Label']) == 0]['Label'].count()
        print("x"+tag, (preds['Label'].min(), preds['Label'].max()), "TP", TP, "FP", FP, "FN", FN, "TN", TN,
              "total", preds['Label'].count(),
              "Prec:", round((TP / (TP + FP) * 100), 2) ,
             "Acc:",  round(((TP + TN) / (P + N))*100, 2) ,
             "F1:", round((2*TP / (2*TP + FP + FN))*100, 2))
#        for j in range((41) if yproc[i] else 0):
        for j in range(3, 41):
            print(tfiles[j], end = " ")
            testdf = (pd.read_csv(tfiles[j]))
            preds = predict_model(model, data=testdf, verbose=True)
            preds['Labelnew'] = round(preds['Label'])
            labels = preds.groupby(['Labelnew'], as_index=False)['Labelnew'].count()
            print(" ".join([str(l) + ":" + str(labels.loc[l][0]) for l in labels.index]), (labels.loc[1][0]/labels.sum()[0]))
            preds[['customer_ID', 'Label']].to_csv(root+"/test/" + tag + "/" + tfiles[j].replace("dtcnt.csv", ".preds"), index=False)
else:
#    k=1
    xfiles = ["x" + str((i*nof)+j) + "13dts.csv" for i in range(int(12/nof)) for j in range(nof)]
    tfiles = ["t" + str((i*nof)+j) + "13dts.csv" for i in range(int(12/nof)) for j in range(nof)]
    print("Loading...", root+"models/" + "lightgbm-x13dtsnew0", end=" ")
    model = load_model(root+"/models/" + "lightgbm-x13dtsnew0")
#    print("Loading...", root+"models/" + "lightgbm-x12-0123reg", end=" 9
#    model = load_model(root+"/models/" + "lightgbm-x12-0123reg")
    mdf=pd.read_csv("mdfdtcnt.csv")
#    for j in range(len(xfiles)):
    for j in range(12):
#        if j < 6:
#            print("Loading...", root+"models/" + "lightgbm-x13dtsnew" + str(j), end=" ")
#            model = load_model(root+"/models/" + "lightgbm-x13dtsnew" + str(j))

#        if j == 6:
#            print("Loading...", root+"models/" + "lightgbm-x12-0123reg", end=" ")
#            model = load_model(root+"/models/" + "lightgbm-x12-0123reg")

        print(xfiles[j], end=" ")
        testdf = pd.read_csv(xfiles[j])
        testdf = pd.concat([mdf, testdf], ignore_index=True)

        preds = predict_model(model, data=testdf, verbose=True)
        preds[['customer_ID', 'target', 'Label']].to_csv(root+"/test/" + xfiles[j].replace(".csv", "new.preds"), index=False)
        P = preds[preds['target']==1]['Label'].count()
        N = preds[preds['target']==0]['Label'].count()
        TP = preds[preds['target']==1][round(preds['Label']) == 1]['Label'].count()
        FP = preds[preds['target']==0][round(preds['Label']) == 1]['Label'].count()
        FN = preds[preds['target']==1][round(preds['Label']) == 0]['Label'].count()
        TN = preds[preds['target']==0][round(preds['Label']) == 0]['Label'].count()
        print((preds['Label'].min(), preds['Label'].max()), "TP", TP, "FP", FP, "FN", FN, "TN", TN,
              "total", preds['Label'].count(),
              "Prec:", round((TP / (TP + FP) * 100), 2) ,
             "Acc:",  round(((TP + TN) / (P + N))*100, 2) ,
             "F1:", round((2*TP / (2*TP + FP + FN))*100, 2))
#        k+=1
#        if k==3:
#            print("Loading...", root+"models/" + "lightgbm-x12-0123reg", end=" ")
#            model = load_model(root+"/models/" + "lightgbm-x12-0123reg")
#    k=1

        if len(predsrange) <= i:
            predsrange.append((preds['Label'].min(), preds['Label'].max()))
        else:
            predsrange[i] = (preds['Label'].min(), preds['Label'].max())

#    for j in range(len(tfiles)):
        print(tfiles[j], end = " ")
        testdf = (pd.read_csv(tfiles[j]))
        testdf=pd.concat([mdf, testdf], ignore_index=True)
        preds = predict_model(model, data=testdf, verbose=True)
        preds['Labelnew'] = round(preds['Label'])
        labels = preds.groupby(['Labelnew'], as_index=False)['Labelnew'].count()
        print("total:",labels.sum()[0], " ".join([str(l) + ":" + str(labels.loc[l][0]) for l in labels.index]), (labels.loc[1][0]/labels.sum()[0]))
        preds[['customer_ID', 'Label']].to_csv(root+"/test/" + tfiles[j].replace(".csv", "new.preds"), index=False)
#        k+=1
#        if k==3:
#            print("Loading...", root+"models/" + "lightgbm-x12-0123reg", end=" ")
#            model = load_model(root+"/models/" + "lightgbm-x12-0123reg")
predsrange

Loading... /home/sadagopan/amex-default-prediction/models/lightgbm-x12-891011reg Transformation Pipeline and Model Successfully Loaded
x891011 (-0.0669040731714942, 1.1353473447921785) TP 15063 FP 2503 FN 3657 TN 59143 total 80366 Prec: 85.75 Acc: 92.34 F1: 83.02
t12-3dtcnt.csv 0:15539 1:4494 0.22432985573803224
t12-4dtcnt.csv 0:15541 1:4491 0.22419129392971246
t12-5dtcnt.csv 0:15525 1:4502 0.22479652469166625
t12-6dtcnt.csv 0:15410 1:4620 0.2306540189715427
t12-7dtcnt.csv 0:15619 1:4412 0.2202585991712845
t12-8dtcnt.csv 0:15467 1:4562 0.2277697338858655
t12-9dtcnt.csv 0:15598 1:4439 0.22154015072116584
t12-10dtcnt.csv 0:15611 1:4420 0.22065798013079727
t12-11dtcnt.csv 0:15515 1:4519 0.2255665368872916
t12-12dtcnt.csv 0:15665 1:4362 0.21780596195136565
t12-13dtcnt.csv 0:15426 1:4609 0.2300474170202146
t12-14dtcnt.csv 0:15565 1:4465 0.22291562656015976
t12-15dtcnt.csv 0:15539 1:4498 0.22448470329889705
t12-16dtcnt.csv 0:15492 1:4536 0.22648292390653085
t12-17dtcnt.csv 0:15479 1:4553 0.2


KeyboardInterrupt



In [8]:
preds
labels.sum()[0]
testdf
#model=None
tfiles

#testdf = (pd.read_csv("t12-40dtcnt.csv"))
#preds = predict_model(model, data=testdf, verbose=True)
#preds['Labelnew'] = round(preds['Label'])
#labels = preds.groupby(['Labelnew'], as_index=False)['Labelnew'].count()
#print(" ".join([str(l) + ":" + str(labels.loc[l][0]) for l in labels.index]), (labels.loc[1][0]/labels.sum()[0]))
preds[['customer_ID', 'Label']].to_csv(root+"/test/" + tag + "/" + "t12-40.preds", index=False)


In [ ]:
preds = []
for r, d, f in os.walk(root+"test"):
        f.sort()
#    if r == "/amex/test":
        for file in f:
            if ".dtcnt.preds" in file :
                preds.append(os.path.join(r, file))
preds[len(preds)-1], len(preds)

In [ ]:
preddf= pd.DataFrame()
rangemin, rangemax = 0, 1
for i in range(12):
    preds = pd.read_csv(root+"/test/x"+ str(i) + ".preds")
    rangemin = preds['Label'].min()
    rangemax = preds['Label'].max()
    rangemin = min(preds['Label'].min(), rangemin)
    rangemax = max(preds['Label'].max(), rangemax)
    preds = pd.read_csv(root+"/test/t"+ str(i) + ".dtcnt.preds")
    rangemin = min(preds['Label'].min(), rangemin)
    rangemax = max(preds['Label'].max(), rangemax)
    print(preds['Label'].min(), preds['Label'].max(), "actual:", rangemin, rangemax)
    preds['Label'] = ( (preds[['Label']] - rangemin) / (rangemax - rangemin) ) * (1 - 0) + 0
    preddf = pd.concat([preddf, preds], ignore_index=True, axis=0)

print("")
preds = pd.read_csv(root+"/test/x12-012dtcnt.preds")
rangemin = preds['Label'].min()
rangemax = preds['Label'].max()
for i in range(41):
    preds = pd.read_csv(root+"/test/t12-"+ str(i) + ".preds")
    rangemin = min(preds['Label'].min(), rangemin)
    rangemax = max(preds['Label'].max(), rangemax)
    print(preds['Label'].min(), preds['Label'].max(), "actual:", rangemin, rangemax)
    preds['Label'] = ( (preds[['Label']] - rangemin) / (rangemax - rangemin) ) * (1 - 0) + 0
    preddf = pd.concat([preddf, preds], ignore_index=True, axis=0)


In [ ]:
preddf= pd.DataFrame()

preds = pd.read_csv(root+"/test/x5.dtcnt.preds")
rangemin, rangemax = preds['Label'].min(), preds['Label'].max()

for i in range(41):
    preds = pd.read_csv(root+"/test/t12-"+ str(i) + ".dtcnt" + ".preds", names=['customer_ID', 'Label'])
    print(preds['Label'].min(), preds['Label'].max())
    rangemin = min(preds['Label'].min(), rangemin)
    rangemax = max(preds['Label'].max(), rangemax)
#    rangemin = preds['Label'].min()
#    rangemax = preds['Label'].max()
#    print(preds['Label'].min(), preds['Label'].max(), "actual:", rangemin, rangemax)
#    preds['Label'] = ( (preds[['Label']] - rangemin) / (rangemax - rangemin) ) * (1 - 0) + 0
    preddf = pd.concat([preddf, preds], ignore_index=True, axis=0)

print(preddf['Label'].min(), preddf['Label'].max(), "actual:", rangemin, rangemax)
preddf['Label'] = ( (preddf[['Label']] - rangemin) / (rangemax - rangemin) ) * (1 - 0) + 0

preddf['Label'].min(), preddf['Label'].max()

In [23]:
#preddf = preddf.groupby(['customer_ID'], as_index=False)['Label'].mean() #.round() #.reset_index()
valids = pd.merge(samples, preddf, on='customer_ID', how='inner')
valids.rename(columns={'prediction': 'target'}, inplace=True)
valids.rename(columns={'Label': 'prediction'}, inplace=True)

#valids['prediction'] = ( (valids[['prediction']] - valids[['prediction']].min()) / (valids[['prediction']].max() - valids[['prediction']].min()) ) * (1 - 0) + 0

valids['prediction'].fillna(0, inplace=True)

print("matching 1's", valids[(valids['target'] == round(valids['prediction'])) & (valids['target'] == 1)].count()[0], valids[(valids['target'] == 1)].count()[0])
print("matching 0's", valids[(valids['target'] == round(valids['prediction'])) & (valids['target'] == 0)].count()[0], valids[(valids['target'] == 0)].count()[0])
print("matching", valids[valids['target'] == round(valids['prediction'])].count()[0], "total", valids.count()[0], "%", valids[valids['target'] == round(valids['prediction'])].count()[0]/valids.count()[0])

matching 1's 0 0
matching 0's 714158 924621
matching 714158 total 924621 % 0.7723791694110343


In [ ]:
## valids[valids['prediction']>= 0.8497913915].count()/793212
#valids['prediction'].min(), valids['prediction'].max()
#pd.DataFrame(
valids.groupby(['prediction', 'target']).count().sort_values(by=["prediction"], ascending=False).head(50)

In [31]:
print("matching", valids[valids['target'] == round(valids['prediction'])].count()[0], "total", valids.count()[0], "%", valids[valids['target'] == round(valids['prediction'])].count()[0]/valids.count()[0])

matching 736896 total 924621 % 0.7969708669822554


In [86]:
valids[(valids['prediction'] <= 1)].sort_values('prediction', ascending=False)
valids['customer_ID']

len(samples[~samples.customer_ID.isin(valids.customer_ID)]['customer_ID'])
#preddf.groupby(['customer_ID'], as_index=False)['Label'].mean()

329

In [24]:
#!mv /amex/submission.csv submission11.csv

valids[['customer_ID', 'prediction']].to_csv('submission.csv', index=False)

In [154]:
800000/924621
38716/(38716+119855)

0.24415561483499504

In [4]:
#preddf= pd.DataFrame()
#for i in range(78):
#    preddf = pd.concat([preddf, pd.read_csv(preds[i], names=['customer_ID', 'Label'], header=None)], ignore_index=True, axis=0)

In [5]:
#preddf['Label'] = preddf['Label'].apply(lambda x: abs(x))
#preddf['Label'] = (preddf['Label'] / preddf['Label'].max())
#preddf['Label'] = preddf['Label'].apply(lambda x: 1 if x>1 else x)
#preddf['Label'] = preddf['Label'].apply(lambda x: 0 if x<0 else x)

#preddf = preddf.groupby(['customer_ID'], as_index=False)['Label'].mean() #.round() #.reset_index()

#pd.DataFrame(preddf)
#preddf['Label'] = preddf.Label.astype("int")
#preddf.head(1)

In [ ]:
valids['prediction'].agg(['min', 'max'])
#731000/924621

In [ ]:
#valids.loc[(valids['prediction']>0.1) & (valids['prediction']< 0.5), 'prediction'] = valids.loc[(valids['prediction']>0.1) & (valids['prediction']< 0.5), 'prediction'] * 0.9

#valids.loc[(valids['prediction']>0.1) & (valids['prediction']< 0.5), 'prediction']

df = (pd.concat([valids[['customer_ID', 'prediction']].rename(columns={'prediction':'target'}), valids[['customer_ID', 'prediction']]], axis='columns')).sort_values('prediction', ascending=False)
#df = (pd.concat([valids[['customer_ID', 'target']], valids[['customer_ID', 'prediction']]], axis='columns')).sort_values('prediction', ascending=False)

#df['correcpred'] = df['prediction'].apply(lambda x: 1 if x> 1 else (0 if x < 0 else x))

df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)

df['random'] = (df['weight'] / df['weight'].sum()).cumsum()

total_pos = (df['target'] * df['weight']).sum()

df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()

df['lorentz'] = df['cum_pos_found'] / total_pos

df['gini'] = (df['lorentz'] - df['random']) * df['weight']

#(df['correcpred'] * df['weight']).sum(), (df['prediction'] * df['weight']).sum()
df['gini'].sum()


#1782137.8168000695/303412.06746732915

#valids[valids['prediction']>1].max()
df#[df['target']>0.9]

In [11]:
predsrange = [0, 0]
for j in range(12):
    print("t" + str(j) + "dtcnt.csv", end = " ")
    preds = pd.read_csv("test/t" + str(j) + ".dtcnt.preds")
    preds['Labelnew'] = round(preds['Label'])
    labels = preds.groupby(['Labelnew'], as_index=False)['Labelnew'].count()
    print(" ".join([str(l) + ":" + str(labels.loc[l][0]) for l in labels.index]))
    predsrange[0] += labels.loc[0][0]
    predsrange[1] += labels.loc[1][0]
print(predsrange, predsrange[1]/sum(predsrange))

predsrange = [0, 0]
for j in range(41):
    print("t12-" + str(j) + "dtcnt.csv", end = " ")
    preds = pd.read_csv("test/t12-" + str(j) + ".preds")
    preds['Labelnew'] = round(preds['Label'])
    labels = preds.groupby(['Labelnew'], as_index=False)['Labelnew'].count()
    print(" ".join([str(l) + ":" + str(labels.loc[l][0]) for l in labels.index]))
    predsrange[0] += labels.loc[0][0]
    predsrange[1] += labels.loc[1][0]
predsrange, predsrange/sum(predsrange)

t0dtcnt.csv 0:5432 1:395
t1dtcnt.csv 0:6997 1:619
t2dtcnt.csv 0:5739 1:1928
t3dtcnt.csv 0:5840 1:2394
t4dtcnt.csv 0:5697 1:2619
t5dtcnt.csv 0:5790 1:2800
t6dtcnt.csv 0:6091 1:3059
t7dtcnt.csv 0:6299 1:3113
t8dtcnt.csv 0:6887 1:3233
t9dtcnt.csv 0:5714 1:3449
t10dtcnt.csv 0:5546 1:3688
t11dtcnt.csv 0:6571 1:4433
[72603, 31730] 0.3041223773877872
t12-0dtcnt.csv 0:15772 1:4264
t12-1dtcnt.csv 0:15601 1:4428
t12-2dtcnt.csv 0:15701 1:4328
t12-3dtcnt.csv 0:15686 1:4347
t12-4dtcnt.csv 0:15716 1:4316
t12-5dtcnt.csv 0:15686 1:4341
t12-6dtcnt.csv 0:15596 1:4434
t12-7dtcnt.csv 0:15738 1:4293
t12-8dtcnt.csv 0:15618 1:4411
t12-9dtcnt.csv 0:15704 1:4333
t12-10dtcnt.csv 0:15724 1:4307
t12-11dtcnt.csv 0:15664 1:4370
t12-12dtcnt.csv 0:15830 1:4197
t12-13dtcnt.csv 0:15539 1:4496
t12-14dtcnt.csv 0:15713 1:4317
t12-15dtcnt.csv 0:15677 1:4360
t12-16dtcnt.csv 0:15671 1:4357
t12-17dtcnt.csv 0:15623 1:4409
t12-18dtcnt.csv 0:15711 1:4319
t12-19dtcnt.csv 0:15669 1:4359
t12-20dtcnt.csv 0:15696 1:4332
t12-21dtcnt.c

([642845, 177443], array([0.78368207, 0.21631793]))